<a href="https://colab.research.google.com/github/deltorobarba/sciences/blob/master/hpc_galaxy_mpi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **N-body Galaxy Simulation with MPI Parallelization**

Supercomputing for astrophysical simulations using MPI (Message Passing Interface), which is a standard parallel computing framework used in supercomputing environments:

In [ ]:
#!/usr/bin/env python
"""
N-body simulation of galaxy formation using MPI for parallelization
This code would typically run on supercomputers with hundreds or thousands of cores
"""

import numpy as np
from mpi4py import MPI
import h5py
import time
import argparse

# Initialize MPI environment
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Parse command line arguments
parser = argparse.ArgumentParser(description='N-body simulation with MPI')
parser.add_argument('--particles', type=int, default=100000, help='Number of particles')
parser.add_argument('--timesteps', type=int, default=1000, help='Number of timesteps')
parser.add_argument('--output', type=str, default='galaxy_sim.h5', help='Output file')
parser.add_argument('--softening', type=float, default=0.1, help='Gravitational softening length')
args = parser.parse_args()

# Constants
G = 6.67430e-11  # Gravitational constant
mass_scale = 1.0e30  # Mass scale (kg)
distance_scale = 3.086e19  # Distance scale (meters, ~ 1 parsec)
time_scale = 3.154e13  # Time scale (seconds, ~ 1 million years)

# Scaled gravitational constant
G_scaled = G * (time_scale**2 * mass_scale) / (distance_scale**3)

def initialize_particles(n_particles, rank, size):
    """Initialize particle positions and velocities for a disk galaxy"""
    np.random.seed(42 + rank)  # Different seed for each process

    # Calculate how many particles this process will handle
    particles_per_process = n_particles // size
    start_idx = rank * particles_per_process
    end_idx = start_idx + particles_per_process if rank < size - 1 else n_particles
    local_n = end_idx - start_idx

    # Initialize arrays for this process's particles
    positions = np.zeros((local_n, 3))
    velocities = np.zeros((local_n, 3))
    masses = np.zeros(local_n)

    # Create disk galaxy with bulge
    for i in range(local_n):
        global_i = start_idx + i  # Global particle index

        if global_i < 0.2 * n_particles:  # Bulge particles
            r = 0.5 * np.random.exponential(scale=0.5)
            theta = np.random.uniform(0, 2*np.pi)
            phi = np.arccos(2 * np.random.random() - 1)

            positions[i, 0] = r * np.sin(phi) * np.cos(theta)
            positions[i, 1] = r * np.sin(phi) * np.sin(theta)
            positions[i, 2] = r * np.cos(phi) * 0.2  # Flattened bulge

            # Bulge velocities - mostly random with some rotation
            v_mag = np.sqrt(G_scaled * 10.0 / r) * 0.7
            v_phi = np.random.uniform(0, 2*np.pi)
            velocities[i, 0] = v_mag * np.cos(v_phi)
            velocities[i, 1] = v_mag * np.sin(v_phi)
            velocities[i, 2] = np.random.normal(0, 0.1)

            masses[i] = np.random.uniform(0.5, 2.0)

        else:  # Disk particles
            r = np.random.exponential(scale=2.0) + 0.5
            theta = np.random.uniform(0, 2*np.pi)

            positions[i, 0] = r * np.cos(theta)
            positions[i, 1] = r * np.sin(theta)
            positions[i, 2] = np.random.normal(0, 0.1)  # Thin disk

            # Calculate circular orbital velocity for disk stability
            v_mag = np.sqrt(G_scaled * 5.0 / r)
            velocities[i, 0] = -v_mag * np.sin(theta)  # Circular orbit
            velocities[i, 1] = v_mag * np.cos(theta)
            velocities[i, 2] = np.random.normal(0, 0.05)

            masses[i] = np.random.uniform(0.1, 1.0)

    return positions, velocities, masses

def calculate_forces(positions, masses, softening):
    """Calculate gravitational forces between particles"""
    n = positions.shape[0]
    forces = np.zeros_like(positions)
    potential_energy = 0.0

    # Direct N-body force calculation - O(n²) complexity
    # In production code, this would use tree methods or FFT for O(n log n) scaling
    for i in range(n):
        # Vectorized force calculation
        dx = positions[i] - positions
        r2 = np.sum(dx**2, axis=1) + softening**2
        inv_r3 = 1.0 / (r2 * np.sqrt(r2))

        # Zero out self-interaction
        inv_r3[i] = 0.0

        # Calculate forces
        f = G_scaled * masses[i] * masses.reshape(-1, 1) * dx.T * inv_r3
        forces[i] += -np.sum(f.T, axis=0)

        # Accumulate potential energy
        potential_energy += -0.5 * G_scaled * masses[i] * np.sum(masses / np.sqrt(r2))

    return forces, potential_energy

def integrate_leapfrog(positions, velocities, masses, dt, softening):
    """Leapfrog integrator for advancing simulation one timestep"""
    # Calculate initial forces
    forces, potential_energy = calculate_forces(positions, masses, softening)

    # Half step in velocity
    velocities += 0.5 * forces / masses.reshape(-1, 1) * dt

    # Full step in position
    positions += velocities * dt

    # Recalculate forces with updated positions
    forces, potential_energy = calculate_forces(positions, masses, softening)

    # Half step in velocity
    velocities += 0.5 * forces / masses.reshape(-1, 1) * dt

    # Calculate kinetic energy
    kinetic_energy = 0.5 * np.sum(masses.reshape(-1, 1) * velocities**2)

    return positions, velocities, potential_energy, kinetic_energy

def save_snapshot(positions, velocities, masses, output_file, timestep):
    """Save simulation snapshot to HDF5 file"""
    # Gather data from all processes
    all_positions = comm.gather(positions, root=0)
    all_velocities = comm.gather(velocities, root=0)
    all_masses = comm.gather(masses, root=0)

    if rank == 0:
        # Concatenate arrays from all processes
        positions_full = np.concatenate(all_positions)
        velocities_full = np.concatenate(all_velocities)
        masses_full = np.concatenate(all_masses)

        # Open or create HDF5 file
        with h5py.File(output_file, 'a') as f:
            # Create group for this timestep
            grp_name = f'timestep_{timestep:06d}'
            if grp_name in f:
                del f[grp_name]  # Remove if exists
            grp = f.create_group(grp_name)

            # Save data
            grp.create_dataset('positions', data=positions_full)
            grp.create_dataset('velocities', data=velocities_full)
            grp.create_dataset('masses', data=masses_full)
            grp.attrs['time'] = timestep * args.timestep

def run_simulation():
    """Run the main simulation loop"""
    if rank == 0:
        print(f"Starting N-body simulation with {args.particles} particles on {size} processes")
        # Create empty output file
        with h5py.File(args.output, 'w') as f:
            f.attrs['n_particles'] = args.particles
            f.attrs['n_timesteps'] = args.timesteps
            f.attrs['softening'] = args.softening

    # Initialize particles
    positions, velocities, masses = initialize_particles(args.particles, rank, size)

    # Main simulation loop
    dt = 0.01  # Time step in simulation units
    save_interval = max(1, args.timesteps // 100)  # Save 100 snapshots

    # Save initial state
    save_snapshot(positions, velocities, masses, args.output, 0)

    start_time = time.time()
    for step in range(1, args.timesteps + 1):
        # Integrate one step
        positions, velocities, potential, kinetic = integrate_leapfrog(
            positions, velocities, masses, dt, args.softening
        )

        # Collect energy information from all processes
        total_potential = comm.reduce(potential, op=MPI.SUM, root=0)
        total_kinetic = comm.reduce(kinetic, op=MPI.SUM, root=0)

        # Print progress
        if rank == 0 and step % (args.timesteps // 10) == 0:
            total_energy = total_potential + total_kinetic
            print(f"Step {step}/{args.timesteps}, "
                  f"E_pot={total_potential:.6e}, E_kin={total_kinetic:.6e}, "
                  f"E_tot={total_energy:.6e}")

        # Save snapshot at regular intervals
        if step % save_interval == 0:
            save_snapshot(positions, velocities, masses, args.output, step)

    end_time = time.time()
    if rank == 0:
        print(f"Simulation completed in {end_time - start_time:.2f} seconds")

if __name__ == "__main__":
    run_simulation()
    MPI.Finalize()